# Chapter 4 - Bedrock APIs: Product Description and Visual Generator

## Overview
This notebook demonstrates how to use Amazon Bedrock for content generation tasks, specifically creating product descriptions and visual content. We'll explore how to leverage foundation models for creative and marketing applications.

## Introduction
This notebook demonstrates how to leverage Amazon Bedrock's foundation models to generate compelling product descriptions and visuals based on product concepts. By combining text generation with image creation capabilities, you can quickly prototype product ideas and create marketing materials

## Prerequisites
- AWS account with Amazon Bedrock access
- Access to Claude 3 Sonnet and Nova Canvas models
- Basic understanding of Python and AWS SDK

## Setup

### Install Required Dependencies

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install Pillow

### Import Libraries

In [ ]:
import json
import os
import sys

import boto3
import botocore

import base64
from PIL import Image
from io import BytesIO

boto3_bedrock = boto3.client('bedrock-runtime')

## Product Concept Definition

### Define Product Idea

In [ ]:
user_input = "Baby Stroller and Scooter Hybrid"

In [ ]:
text_prompt = f"""
You are an assistant designed to help product teams craft compelling and informative product descriptions. Your task is to generate detailed, realistic, and easy-to-understand product descriptions based on the user's input about their product idea.

Your product descriptions should be realistic, clear, concise, and effectively convey the key features, benefits, and unique selling points of the product. Your goal is to create descriptions that are engaging, persuasive, and help potential customers understand the value and functionality of the product.

Please use the user input user to understand their product concept and tailor the descriptions accordingly. Your output should be of a professional quality, reflecting the specific needs and requirements of the product team.

Here is the product idea - ```{user_input}```
"""

## Generate Product Description

### Configure and Call Claude 3 Sonnet

In [ ]:
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
              "type": "text", 
              "text": text_prompt
          }
        ]
      }
    ],
    "temperature":0.5,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
})

### Generate Product Description with Claude 3 Sonnet

In [ ]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

try:
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    print(response_body.get('content')[0]['text'])
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

## Generate Product Visual

### Configure and Call Nova Canvas

In [ ]:
# Nova Canvas Image Generation
nova_prompt = f"""
Create a professional product photo of a {user_input}.
The image should show a sleek, modern design with clean lines and premium materials.
Show the product in a bright, well-lit studio setting with a clean white background.
The hybrid design should be clearly visible, demonstrating both stroller and scooter functionality.
Include safety features like harnesses and brakes. Make it look realistic, usable, innovative and family-friendly.
High quality, professional product photography style.
"""

nova_body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": nova_prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 0
        }
    })

In [ ]:
# Generate image with Nova Canvas
nova_model_id = 'amazon.nova-canvas-v1:0'

try:
    nova_response = boto3_bedrock.invoke_model(
        body=nova_body,
        modelId=nova_model_id,
        accept='application/json',
        contentType='application/json'
    )
    
    nova_response_body = json.loads(nova_response.get('body').read())
    nova_image_data = nova_response_body['images'][0]
    
    # Decode and display the image
    nova_image_bytes = base64.b64decode(nova_image_data)
    nova_image = Image.open(BytesIO(nova_image_bytes))
    
    print("Nova Canvas generated image:")
    display(nova_image)
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\n"
              "To troubleshoot this issue please refer to the following resources.\n"
              "https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\n"
              "https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error

## Conclusion

This notebook demonstrates a powerful workflow for product concept development and marketing content creation using foundation models. By combining Claude 3 Sonnet's sophisticated text generation with Nova Canvas's visual creation capabilities, we've shown how you can:

1. **Rapidly Prototype Ideas**: Transform simple product concepts into detailed descriptions and visual representations in minutes rather than days or weeks.

2. **Generate Marketing Content**: Create professional-quality product descriptions suitable for websites, catalogs, and other marketing materials.

3. **Visualize Concepts**: Generate realistic product images without photography or complex design work.

4. **Iterate Efficiently**: Quickly refine both descriptions and visuals by adjusting prompts based on feedback.

This approach has numerous practical applications:
- Product development teams exploring new concepts
- Marketing teams creating preliminary materials
- Startups developing pitch materials
- E-commerce businesses building product listings

The combination of text and image generation demonstrates how foundation models can enhance creativity and productivity in product development and marketing workflows. By leveraging these capabilities through Amazon Bedrock's API, businesses can significantly accelerate their ideation-to-market process.

For further development, you could explore:
- Adding variation controls to generate multiple product versions
- Creating targeted descriptions for different audience segments
- Generating complementary assets like lifestyle images or usage scenarios
- Implementing feedback loops to iteratively improve outputs based on user preferences

This demonstrates just one of many possible workflows that combine multiple foundation models to solve complex business problems through Amazon Bedrock.